In [1]:
# %pip install boto3==1.34.93 ipython --quiet

%load_ext autoreload
%autoreload 2

In [2]:
# import boto3
import json, re
# %time
# session = boto3.session.Session()
# bedrock_client = session.client(
#     service_name='bedrock-runtime',
#     region_name='eu-west-3',
# )

# # Model id and Claude config
# model_id = "anthropic.claude-3-sonnet-20240229-v1:0"
# claude_config = {
#     'max_tokens': 4096,
#     'temperature': 0, 
#     'anthropic_version': '',
#     'stop_sequences': ['Human:']
# }

In [3]:
# AO
cdc_json_file_path = f"/workspace/outputs/2023DOS0550696/json/ao/formatted_json_cdc.json"
rec_json_file_path = f"/workspace/outputs/2023DOS0550696/json/ao/formatted_json_rec.json"
# CV
entreprise = "cgi"
cv_json_file_path = f"/workspace/outputs/2023DOS0550696/json/{entreprise}/formatted_json_cv.json"

In [4]:
# Open the JSON file and load its contents
with open(cdc_json_file_path, "r") as file:
    cdc_json_data = json.load(file)
with open(cv_json_file_path, "r") as file:
    cv_json_data = json.load(file)
with open(rec_json_file_path, "r") as file:
    rec_json_data = json.load(file)

# Now you can work with the loaded JSON data
print(cdc_json_data)
print(cv_json_data)

# Add you json inputs here
REQUIREMENTS = cdc_json_data
RESUME = cv_json_data

{'objectif_de_la_mission': ["La mission objet de cette consultation vise à décharger les équipes SNCF dans trois domaines concernant les services bureautiques et digitaux au sens large : - La gestion courante des sollicitations et demandes ; - L'assistance et le conseil aux utilisateurs ; - La gestion des stocks, de l'approvisionnement à la mise au rebus en passant par la remise en main.", 1], 'duree_de_la_mission': [210, 3], 'nombre_de_profils': [1, 2], 'types_de_profils': [{'type': ['701.E.PAR - Support Méthode, qualité et Sécurité Expert - ouvert à des profils Senior (701.S.PAR)', 2]}], 'taches_et_responsabilites': [{'tache_et_responsabilite': ['Traiter les demandes des agents G&C reçues via le portail interne Digital Office', 1]}, {'tache_et_responsabilite': ["Prendre en charge les déclarations d'incidents ou d'anomalie de fonctionnement bureautique quand les processus standard des porteurs de services ne suffisent pas", 1]}, {'tache_et_responsabilite': ['Alerter / escalader en int

In [5]:
CRITERIAS  = rec_json_data['sous_criteres_techniques_et_ponderation']

for critere_json in rec_json_data['sous_criteres_techniques_et_ponderation']:
    print(critere_json['critere'])

['Qualité de la réponse (Compréhension du besoin, le cas échéant outils et méthodologie proposés)', 1]
['Nombre et pertinence des expériences similaires', 1]
['Compétences techniques', 1]
['Connaissance du domaine fonctionnel', 1]
["Expériences similaires en accompagnement au changement, actions pédagogiques illustrant la capacité d'écoute, de reformulation et de synthèse", 1]


In [6]:
SCALE = {
    0: "Ne répond pas au besoin",
    1: "Ne répond pas suffisamment au besoin",
    2: "Répond partiellement au besoin",
    3: "Répond correctement au besoin",
    4: "Très bonne réponse",
    5: "Réponse au-delà des attentes"
}


In [7]:
# prompt = f"""
# Vous êtes un assistant d'analyse des compétences techniques de CV dans le cadre de réponse à appel
# d'offre du groupe SNCF. Votre tâche est d'évaluer un CV par rapport à un ensemble de critères
# techniques pondérés fournis.

# Voici les informations dont vous disposez :

# <Requirements>
# {REQUIREMENTS}
# </Requirements>

# <Criteria>
# {CRITERIAS}
# </Criteria>

# <Resume>
# {RESUME}
# </Resume>

# <Scale>
# {SCALE}
# </Scale>
# """ + """
# Commencez par lire attentivement les exigences et les critères. Ensuite, analysez le CV par rapport
# à chaque critère. Pour chaque critère, fournissez d'abord une justification détaillée de la note que
# vous attribuez au candidat. Utilisez les informations du CV pour expliquer pourquoi vous pensez que
# le candidat mérite cette note pour ce critère spécifique.

# Après avoir fourni la justification, donnez une note numérique entre 1 et 5 en vous basant sur
# l'échelle de notation fournie.

# Une fois que vous avez analysé tous les critères, produisez un JSON avec les notes et les
# justifications pour chaque critère.

# Le format du JSON doit être le suivant :

# {
# 'critère 1': {
# 'justification': 'Justification détaillée pour la note',
# 'score': X
# },
# 'critère 2': {
# 'justification': 'Justification détaillée pour la note',
# 'score': Y
# },
# ...
# }

# Remplacez "critère 1", "critère 2", etc. par les noms réels des critères. Assurez-vous que les
# justifications sont claires et détaillées, et que les scores sont conformes à l'échelle de notation
# fournie.

# <scratchpad>
# Vous pouvez utiliser cette zone pour prendre des notes et organiser vos pensées avant de produire le
# JSON final.
# </scratchpad>

# <json_output>
# Écrivez votre sortie JSON ici.
# </json_output>
# """

In [8]:
# def prepare_bedrock_request(messages, claude_config):
#     return {
#         'messages': messages,
#         **claude_config,
#     }
# def extract_bedrock_response(response):
#     body = json.loads(response['body'].read().decode('utf-8'))
#     return body['content'][0]['text']

def pretty_print(message):
    print('\n\n'.join('\n'.join(line.strip() for line in re.findall(r'.{1,100}(?:\s+|$)', paragraph.strip('\n'))) for paragraph in re.split(r'\n\n+', message)))

In [9]:
# %%time
# messages = [
#     {"role": "user", "content": [
#         {"type": "text", "text": prompt},
#         ]
#     }
# ]

# body = prepare_bedrock_request(messages, claude_config)
# response = bedrock_client.invoke_model(modelId=model_id, body=json.dumps(body))
# formated_response = extract_bedrock_response(response)

In [10]:
from dag_nao.modules.technical_notation.single_resume_simple_call_comparator.llm_comparator import LLMComparator

llm_comparator = LLMComparator()
formated_response, formated_json_response, score = llm_comparator.call_comparator(requirements=REQUIREMENTS, criterias=CRITERIAS, resume=RESUME, scale=SCALE)

Note technique : 4.3


In [11]:
pretty_print(formated_response)

Voici mon analyse détaillée du CV par rapport aux critères fournis :

<scratchpad>
Critère 1 : Qualité de la réponse (Compréhension du besoin, le cas échéant outils et méthodologie
proposés)
- Le CV ne fournit pas d'informations spécifiques sur la compréhension du besoin ou les
outils/méthodologies proposés pour cette mission. Il décrit principalement les expériences passées
du candidat.
=> Score : 2 (Répond partiellement au besoin)

Critère 2 : Nombre et pertinence des expériences similaires
- Le candidat a une expérience de 6 ans en tant que "Support Méthode, qualité et Sécurité" chez
Avancial, qui semble très pertinente pour cette mission.
- Il a également une expérience de 6 ans comme "Responsable qualité/méthode" chez SFR, également
pertinente.
- Ses autres expériences comme "Administration et gestion des utilisateurs" à la SNCF, "Support
technique" chez Citroën et "Micro-réseau" chez Canon sont aussi en lien avec les tâches requises.
=> Score : 5 (Réponse au-delà des attentes)

C

In [12]:
formated_json_response

{'Qualité de la réponse (Compréhension du besoin, le cas échéant outils et méthodologie proposés)': {'justification': "Le CV ne fournit pas d'informations spécifiques sur la compréhension du besoin ou les outils/méthodologies proposés pour cette mission. Il décrit principalement les expériences passées du candidat.",
  'score': 2},
 'Nombre et pertinence des expériences similaires': {'justification': 'Le candidat a une expérience de 6 ans en tant que "Support Méthode, qualité et Sécurité" chez Avancial, qui semble très pertinente pour cette mission. Il a également une expérience de 6 ans comme "Responsable qualité/méthode" chez SFR, également pertinente. Ses autres expériences comme "Administration et gestion des utilisateurs" à la SNCF, "Support technique" chez Citroën et "Micro-réseau" chez Canon sont aussi en lien avec les tâches requises.',
  'score': 5},
 'Compétences techniques': {'justification': 'Le CV montre une solide expérience avec les parcs bureautiques Windows/Mac, la ges

In [13]:
CRITERIAS

[{'critere': ['Qualité de la réponse (Compréhension du besoin, le cas échéant outils et méthodologie proposés)',
   1],
  'ponderation': [10, 1]},
 {'critere': ['Nombre et pertinence des expériences similaires', 1],
  'ponderation': [30, 1]},
 {'critere': ['Compétences techniques', 1], 'ponderation': [30, 1]},
 {'critere': ['Connaissance du domaine fonctionnel', 1],
  'ponderation': [20, 1]},
 {'critere': ["Expériences similaires en accompagnement au changement, actions pédagogiques illustrant la capacité d'écoute, de reformulation et de synthèse",
   1],
  'ponderation': [10, 1]}]

In [14]:
# Create a dictionary to store the aggregated scores
aggregated_scores = 0
criteria_scores = formated_json_response
criteria_weights = CRITERIAS
# Iterate through each criterion and its weight
for criterion in criteria_weights:
    critere_name = criterion['critere'][0]
    weight = criterion['ponderation'][0] / 100

    # Iterate through the scores and justifications
    for key, value in criteria_scores.items():
        if key == critere_name:
            # Aggregate the weighted score
            aggregated_scores += value['score'] * (weight)

print(f"Note technique : {aggregated_scores}")

Note technique : 4.3
